In [7]:
import torch

def od_collate_fn(batch):
    """
    Datasetから取り出すアノテーションデータのサイズが画像ごとに異なる。
    画像内の物体数が２個であれば、(2,5)というサイズになるが、３個であれば(3,5)というサイズになる。
    この変化に対応するDataloaderを作成するためにカスタマイズしたcollate_fnを作成する。
    collate_fnはPytorchでリストからmini-batchを作成する関数である。
    ミニバッチ分の画像が並んでいるリスト変数batchにミニバッチ番号を指定する次元を先頭に１つ追加してリストの形を変形する。
    """
    
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0]) #sample[0]は画像img
        targets.append(torch.FloatTensor(sample[1])) #sample[1]はアノテーションgt
        
        
    #imgsはミニバッチサイズのリストになっている
    #リストの要素はtorch.Size([3,300,300])
    #このリストをtorch.Size([batch_num, 3, 300, 300])のテンソルに変換する
    imgs = torch.stack(imgs, dim=0)
    
    #*targetsはアノテーションデータの正解であるgtのリスト
    #*リストのサイズはミニバッチサイズ
    #*リストtargetsの要素は[n, 5]
    #*nは画像ごとに異なり、画像内にあるオブジェクトの数になる。
    #*5は[xmin,ymin,xmax,ymax,class,index]
    return imgs, targets

In [8]:
import myUtilsData as mUD

color_mean = (104, 117, 123) #(BGR)の色の平均値
input_size = 300  #画像のサイズを300*300にする

rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = mUD.make_datapath_list(rootpath)

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

train_dataset = mUD.VOCDataset(train_img_list, train_anno_list, phase="train", transform = mUD.DataTransform(input_size, color_mean), transform_anno = mUD.Anno_xml2list(voc_classes))
val_dataset = mUD.VOCDataset(val_img_list, val_anno_list, phase="val", transform = mUD.DataTransform(input_size, color_mean), transform_anno = mUD.Anno_xml2list(voc_classes))


In [9]:
import torch.utils.data as data
batch_size=4

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=mUD.od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=mUD.od_collate_fn)

#辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

#動作確認
batch_iterator = iter(dataloaders_dict["val"]) #イテレータに変換
images, targets = next(batch_iterator) #1番目の要素を取り出す
print(images.size())  #torch.Size([4,3,300,300])
print(len(targets))
print(targets[1].size()) #ミニバッチのサイズのリスト、各要素は[n,5]、nは物体数

tvmonitor
train
person
boat
cow
cow
torch.Size([4, 3, 300, 300])
4
torch.Size([2, 5])
